In [27]:
import psycopg2
import configparser

import pandas as pd

In [28]:
# lecture du fichier de conf ou est contenu le mot de passe
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [29]:
password = config['credentials']['password']

In [30]:
# parametrer la connection a la bdd

In [82]:
conn = psycopg2.connect(
    host="localhost",       
    database="Brief-02-Alternance", 
    user="postgres",    
    password=password, 
    port="5432"               
)

un connecteur est : ...

In [83]:
cur =  conn.cursor()

In [33]:
# creation des tables
create_ventes = '''
CREATE TABLE IF NOT EXISTS vente (
    product_id VARCHAR(50),
    date TIMESTAMP,
    session_id VARCHAR(50) PRIMARY KEY,
    client_id VARCHAR(50),
    quantity_sold INTEGER
);
'''
cur.execute(create_ventes)
conn.commit()

In [54]:
# creation table client
create_clients = '''
CREATE TABLE IF NOT EXISTS clients (
    client_id VARCHAR(50) PRIMARY KEY,
    sex VARCHAR(50),
    birth INTEGER
);
'''
cur.execute(create_clients)
conn.commit()


In [84]:
# Création de la table produits
create_produits = '''
CREATE TABLE IF NOT EXISTS produits (
    product_id VARCHAR(50) PRIMARY KEY,
    category VARCHAR(50),
    sub_category VARCHAR(50),
    price FLOAT,       
    stock_quantity INTEGER       
);
'''


In [80]:
cur.execute(create_produits)
conn.commit()

ALIMENTATION DES TABLE A PARTIR DES CSV

In [40]:
# IMPORT DES CSV DANS DES DATAFRAME
clients = pd.read_csv('data/clients.csv',sep=';')
produits = pd.read_csv('data/produits_sous-categorie.csv',sep=';')
ventes = pd.read_csv('data/ventes.csv',sep=';')

In [55]:
# Insérer les données dans la table clients
for index, row in clients.iterrows():
    insert_query = '''
    INSERT INTO clients (client_id, sex, birth)
    VALUES (%s, %s, %s)
    ON CONFLICT (client_id) DO NOTHING;  -- Pour éviter les erreurs en cas de doublons
    '''
    cur.execute(insert_query, (row['client_id'], row['sex'], row['birth']))

# Valider l'insertion des données pour les clients
conn.commit()
print("Données insérées avec succès dans la table 'clients' !")

Données insérées avec succès dans la table 'clients' !


In [56]:
# Convertir la colonne birth de INTEGER à TIMESTAMP
alter_table_query = '''
ALTER TABLE clients
ALTER COLUMN birth TYPE TIMESTAMP WITHOUT TIME ZONE
USING TO_TIMESTAMP(birth::text, 'YYYY');
'''
cur.execute(alter_table_query)
conn.commit()
print("Le type de la colonne 'birth' a été modifié avec succès !")

Le type de la colonne 'birth' a été modifié avec succès !


In [61]:
# Supprimer la contrainte de clé primaire
alter_table_query = '''
ALTER TABLE vente DROP CONSTRAINT vente_pkey;
'''
cur.execute(alter_table_query)
conn.commit()

# Ensuite, vous pouvez réinsérer vos données comme précédemment
for index, row in ventes.iterrows():
    insert_query = '''
    INSERT INTO vente (product_id, date, session_id, client_id, quantity_sold)
    VALUES (%s, %s, %s, %s, %s);
    '''
    cur.execute(insert_query, (
        row['product_id'],
        row['date'],
        row['session_id'],
        row['client_id'],
        row['quantity_sold']
    ))

conn.commit()
print("Données insérées avec succès dans la table 'vente' !")

Données insérées avec succès dans la table 'vente' !


In [86]:
# Insérer les données dans la table produits
for index, row in produits.iterrows():
    insert_query = '''
    INSERT INTO produits (product_id, category, sub_category, price, stock_quantity)
    VALUES (%s, %s, %s, %s, %s)
    ON CONFLICT (product_id) DO NOTHING;
    '''
    
    # Convertir le prix de string avec virgule en float
    price = float(row['price'].replace(',', '.'))
    
    cur.execute(insert_query, (
        row['product_id'],
        row['category'],
        row['sub_category'],
        price,
        int(row['stock_quantity'])
    ))

# Valider l'insertion des données pour les produits
conn.commit()
print("Données insérées avec succès dans la table 'produits' !")

Données insérées avec succès dans la table 'produits' !


### quelques requetes sql